In [1]:
import pandas as pd

# Install scikit-learn
%pip install scikit-learn
# Install openpyxl
%pip install openpyxl


from sklearn.model_selection import (
    train_test_split,
)  # used to split your dataset into two parts: a training set and a testing set.
from sklearn.linear_model import (
    LinearRegression,
)  # used to create a linear regression model.
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    mean_absolute_percentage_error,
)  # evaluate the performance of machine learning models
import numpy as np


%pip install matplotlib

import matplotlib.pyplot as plt
from datetime import timedelta



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Load dataset
df = pd.read_excel('Gold_data_filtered.xlsx')

In [3]:
# Prepare the data (ensure the date column is in datetime format)
df['date'] = pd.to_datetime(df['date'])


In [4]:
# Display the first few rows
print(df.head())

# Check for any missing values
print(df.isnull().sum())

        date         open         high          low        close  volume
0 2019-03-01  1312.199951  1312.199951  1288.900024  1296.400024     110
1 2019-03-04  1285.500000  1287.000000  1281.900024  1284.800049      28
2 2019-03-05  1287.000000  1287.000000  1282.000000  1282.000000     114
3 2019-03-06  1287.099976  1287.099976  1282.599976  1284.900024      14
4 2019-03-07  1281.000000  1284.599976  1281.000000  1283.800049      11
date      0
open      0
high      0
low       0
close     0
volume    0
dtype: int64


In [5]:
# Define the features (X) and the target (y)
X = df[['open', 'high', 'low', 'volume']]
y = df['close']